In [1]:
import plotly.graph_objects as go
import pandas as pd

import matplotlib.pyplot as plt
import networkx as nx

# reading input log file
# import ruamel.yaml
import yaml
import re
import ast

import math
import sys
import traceback
from csv import excel
from collections import Counter
from scipy import stats

stat_path="../stat_to_sankey/example_stat/1p9f9s_run"
iamge_path=f"{stat_path}/images"




In [2]:
import os
import re

def search_files_with_name(directory, pattern):
    file_list = []
    for root, dirs, files in os.walk(directory):
        for file in files:
            if re.search(pattern, file):
                file_list.append(os.path.join(root, file))
                #print(os.path.join(root, file))
    return file_list

In [3]:
vfd_files = search_files_with_name(stat_path, "vfd")
# vfd_files = vfd_files[0:1]
print(vfd_files)


def load_vfd_yaml(vfd_files):
    ret_dict = {}
    tmp_dict = {}
    for f in vfd_files:
        with open(f, "r") as stream:
            print(f"loading {f}")
            try:
                tmp_dict = yaml.safe_load(stream)
                # print(tmp_dict)
            except yaml.YAMLError as exc:
                print(exc)
            ret_dict[f] = tmp_dict
    return ret_dict

vfd_dict = load_vfd_yaml(vfd_files)
# print(vfd_dict)

print("loading yaml done")

['../stat_to_sankey/example_stat/1p9f9s_run/98522_vfd-data-stat-dl.yaml', '../stat_to_sankey/example_stat/1p9f9s_run/168881_vfd-data-stat-dl.yaml', '../stat_to_sankey/example_stat/1p9f9s_run/77100_vfd-data-stat-dl.yaml', '../stat_to_sankey/example_stat/1p9f9s_run/167838_vfd-data-stat-dl.yaml', '../stat_to_sankey/example_stat/1p9f9s_run/5904_vfd-data-stat-dl.yaml', '../stat_to_sankey/example_stat/1p9f9s_run/11179_vfd-data-stat-dl.yaml', '../stat_to_sankey/example_stat/1p9f9s_run/75190_vfd-data-stat-dl.yaml', '../stat_to_sankey/example_stat/1p9f9s_run/170958_vfd-data-stat-dl.yaml', '../stat_to_sankey/example_stat/1p9f9s_run/157121_vfd-data-stat-dl.yaml']
loading ../stat_to_sankey/example_stat/1p9f9s_run/98522_vfd-data-stat-dl.yaml
loading ../stat_to_sankey/example_stat/1p9f9s_run/168881_vfd-data-stat-dl.yaml
loading ../stat_to_sankey/example_stat/1p9f9s_run/77100_vfd-data-stat-dl.yaml
loading ../stat_to_sankey/example_stat/1p9f9s_run/167838_vfd-data-stat-dl.yaml
loading ../stat_to_sankey

In [4]:
op_range_shift_dict = {
    "75190":0,
    "5904":7208,
    "77100":12689,
    "168881":15641,
    "98522":30542,
    "170958":31599,
    "11179":34399,
    "157121":37801,
    "167838":91681,
}

# get the I/O range
for k,stat in vfd_dict.items():
    # print(f"VFD Stat File: {k}")
    task_max_io = 0
    prev_file_max_io = 0
    file_op_range_dict = {}
    for li in stat:
        file_key = list(li.keys())[0]
        if 'file' in file_key:
            # file_stat = list(li.values())[0]
            file_stat = li[file_key]
            file_name = file_stat['file_name']
            file_max_io = 0
            file_min_io = prev_file_max_io
            file_max_blob = 0
            try:
                h5mDraw = file_stat['data']['H5FD_MEM_DRAW']
                
                if h5mDraw['read_cnt'] > 0:
                    # print(f"blobs here: {[val[1] for val in h5mDraw['read_ranges'].values()]}")
                    max_blob = max([val[1] for val in h5mDraw['read_ranges'].values()])
                    max_io = max(h5mDraw['read_ranges'].keys())
                    min_io = min(h5mDraw['read_ranges'].keys())
                    if max_blob > file_max_blob: file_max_blob = max_blob
                    if max_io > task_max_io: task_max_io = max_io
                    if max_io > file_max_io: file_max_io = max_io
                    if min_io < file_min_io: file_min_io = min_io
                    
                if h5mDraw['write_cnt'] > 0:
                    # print(f"blobs here: {[val[1] for val in h5mDraw['write_ranges'].values()]}")
                    max_blob = max([val[1] for val in h5mDraw['write_ranges'].values()])
                    max_io = max(h5mDraw['write_ranges'].keys())
                    min_io = min(h5mDraw['read_ranges'].keys())
                    if max_blob > file_max_blob: file_max_blob = max_blob
                    if max_io > task_max_io: task_max_io = max_io
                    if max_io > file_max_io: file_max_io = max_io
                    if min_io < file_min_io: file_min_io = min_io
                
                for meta_type, meta_stats in file_stat['metadata'].items():
                    if file_stat['metadata'][meta_type]['read_cnt'] > 0:
                        max_blob = max([val[1] for val in file_stat['metadata'][meta_type]['read_ranges'].values()])
                        max_io = max(file_stat['metadata'][meta_type]['read_ranges'].keys())
                        min_io = min(file_stat['metadata'][meta_type]['read_ranges'].keys())
                        if max_blob > file_max_blob: file_max_blob = max_blob
                        if max_io > task_max_io: task_max_io = max_io
                        if max_io > file_max_io: file_max_io = max_io
                        if min_io < file_min_io: file_min_io = min_io
                    if file_stat['metadata'][meta_type]['write_cnt'] > 0:
                        max_blob = max([val[1] for val in file_stat['metadata'][meta_type]['write_ranges'].values()])
                        max_io = max(file_stat['metadata'][meta_type]['write_ranges'].keys())
                        min_io = min(file_stat['metadata'][meta_type]['write_ranges'].keys())
                        if max_blob > file_max_blob: file_max_blob = max_blob
                        if max_io > task_max_io: task_max_io = max_io
                        if max_io > file_max_io: file_max_io = max_io
                        if min_io < file_min_io: file_min_io = min_io
                        
            except:
                # Must handle the case where the file is opened but no I/O is performed(recorded?)
                # print(f"Error in {k} stat {file_stat}")
                # print(f"Error in {k} stat ...")
                pass
            
            if file_name in file_op_range_dict.keys():
                file_op_range_dict[file_name]['io_ranges'] .append((file_min_io, file_max_io))
                if file_max_blob != file_op_range_dict[file_name]['blob_reanges'][1]:
                    file_op_range_dict[file_name]['blob_reanges'] = (0, file_max_blob)
                    
            else:
                file_op_range_dict[file_name] = {}
                file_op_range_dict[file_name]['io_ranges'] = [(file_min_io, file_max_io)]
                file_op_range_dict[file_name]['blob_reanges'] = (0, file_max_blob)
                # file_op_range_dict[file_name] = [(file_min_io, file_max_io)]
            
            prev_file_max_io = file_max_io

    file_name = k.split('/')[-1]
    task_id = file_name.split('_')[0]
    # print(f"{task_id} Max IO IDX: {task_max_io}")
    range_shift = op_range_shift_dict[task_id]
    for file_name, ranges_val in file_op_range_dict.items():
        for range in ranges_val['io_ranges']:
            print(f"  - op_count_range: [{range_shift+range[0]}, {range_shift+range[1]}]")
            print(f"    prefetch:")
            print(f"      - bucket: \"{file_name}\"")
            print(f"        promote_blobs: [{ranges_val['blob_reanges'][0]}, {ranges_val['blob_reanges'][1]}]")
        
        # print(f"    - file_name: \"{file_name}\"")
        # for range in ranges_val['io_ranges']:
        #     print(f"      - file_op_range: [{range_shift+range[0]}, {range_shift+range[1]}]")
        # print(f"      - file_blob_range: [{ranges_val['blob_reanges'][0]}, {ranges_val['blob_reanges'][1]}]")
    
    # print("\n")

  - op_count_range: [30542, 30707]
    prefetch:
      - bucket: "trackstats_sparse_20150506.0000_20150506.0800.nc"
        promote_blobs: [0, 0]
  - op_count_range: [30542, 30857]
    prefetch:
      - bucket: "trackstats_sparse_20150506.0000_20150506.0800.nc"
        promote_blobs: [0, 0]
  - op_count_range: [30707, 30542]
    prefetch:
      - bucket: "mcs_tracks_20150506.0000_20150506.0800.nc"
        promote_blobs: [0, 0]
  - op_count_range: [15641, 15713]
    prefetch:
      - bucket: "tracknumbers_20150506.0000_20150506.0800.nc"
        promote_blobs: [0, 0]
  - op_count_range: [15641, 29114]
    prefetch:
      - bucket: "tracknumbers_20150506.0000_20150506.0800.nc"
        promote_blobs: [0, 0]
  - op_count_range: [15713, 15906]
    prefetch:
      - bucket: "cloudid_20150506_000000.nc"
        promote_blobs: [0, 7]
  - op_count_range: [15906, 17343]
    prefetch:
      - bucket: "cloudid_20150506_010000.nc"
        promote_blobs: [0, 7]
  - op_count_range: [17343, 18795]
    